# Lab Assignment Two: Exploring Text Data

## 1. Business Understanding

Short Message Service, also known as SMS, is the one of the largest communication standards in the world. SMS is platform independent, and used by most mobile telephone companies in the world. Even though SMS reached its peak of 3.5 billion active users at the end of 2010, its legacy has not diminished. SMS shaped and inspired a revolution of text messaging web and mobile apps in the years since; the world's currently most famous messenger clients such as iMessage, WhatsApp, Facebook Messenger, and WeChat all drew inspiration from SMS.

In 2008, over 70 billion SMS texts were sent *per month* in the US alone. Unfortunately, a large portion of these texts are sent as unwanted, unwarranted advertisements. These unsolicited messages are called **spam**. Spam can be dangerous, because even though most tech savvy individuals can discern a real, genuine message, from malicious advertisements, many portions of the population cannot (especially the elderly and not tech savvy). Thus, it is important to accurately automate the process of filtering spam SMS from a user's phone in the first place.

The dataset we chose is an SMS Spam Collection Dataset, where each of the 5,574 messages are tagged according to being **spam**, or **ham** (legitimate). This data was collected in the first place in order to properly identify which words are commonly associated with spam messages. The document contains over 90,000 words. Once we begin modelling, our prediction algorithm would need to perform to at least a 50% usage rate in order to beat random. However, in an ideal world in which an automated system should filter out all messages, our team's personal goal would be to beat at least 90% to ensure a clean messaging client for all.

---

Link to Dataset: https://www.kaggle.com/uciml/sms-spam-collection-dataset/kernels

--- 

## 2. Data Encoding As String

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### 2.1 Data Input as Strings

### 2.2 Data Quality Verification

### 2.3 Bag of Words Representation

### 2.4 tf-idf Representation

## 3. Data Visualization

### 3.1 Statistical Summaries Visualized

### 3.2 Most Common Relevant Words

## 4. Word Cloud